In [ ]:
from cedne import utils
import matplotlib.pyplot as plt
import numpy as np
import copy

In [ ]:
import_parameters = {'style':'cook', 'sex':'male'}

In [ ]:
ntype = ['SENSORY NEURONS', 'INTERNEURONS', 'MOTOR NEURONS']
facecolors = ['#FF6F61', '#FFD700', '#4682B4']
ntype_pairs = [(nt1, nt2) for nt1 in ntype for nt2 in ntype]
colors= plt.cm.magma(np.linspace(0,1,len(ntype_pairs)))
type_color_dict = {p:color for (p,color) in zip(ntype_pairs, colors)}

In [ ]:
w = utils.makeWorm('male', import_parameters=import_parameters, chem_only=True)

In [ ]:
nn_male = w.networks['Neutral']

In [ ]:
utils.loadNeurotransmitters(nn_male, sex='Male')

In [ ]:
triad_motifs = utils.return_triads()
motif = triad_motifs['030T']
motif = utils.nx.relabel_nodes(motif, {1:1, 2:3, 3:2})

In [ ]:
mappings = [(2,1), (3,1)]
num_graphs = {}
# conn_types = {mapping:{n:0 for n in ntype_pairs} for mapping in mappings}
ntype_motif = {}
max_chain_length = 6
for mapping in mappings:
    num_graphs[mapping] = []
    ntype_motif[mapping] = {}
    for l in range(1,max_chain_length+1):
        hm = utils.make_hypermotifs(motif, l, [mapping])
        for m in hm.nodes:
            ntype_motif[mapping][m] = {n:0 for n in ntype}
        all_ffgs = nn_male.search_motifs(hm)
        num_graphs[mapping].append(len(all_ffgs))
        for sub in all_ffgs:
            for motif_edge, network_edge in sub.items():
                if all(n.type in ntype for n in network_edge):
                    for m,n in zip(motif_edge, network_edge):
                        ntype_motif[mapping][m][n.type] +=1
        #             conn = tuple(sorted([network_edge[0].type, network_edge[1].type]))
        #             conn_types[mapping][conn] +=1

In [ ]:
motif_fractions_actual = {}
for mapping in mappings:
    motif_fractions_actual[mapping] = np.array(num_graphs[mapping])/(np.array(num_graphs[mappings[0]]) + np.array(num_graphs[mappings[1]]))
colors = ['gray', 'purple']
f, ax = plt.subplots(figsize=(2.5,2.5), layout='constrained')
for m, mapping in enumerate(mappings):
    ax.scatter(np.arange(1,len(num_graphs[mapping])+1),motif_fractions_actual[mapping] , color=colors[m], label=mapping)
ax.set_xticks(np.arange(1,max_chain_length+1))
ax.set_yticks((0,0.5,1))
# ax.yaxis.major.formatter._useMathText = True
ax.set_ylabel("fraction subnetworks with motif")
ax.set_xlabel("Number of chained FFLs")
utils.simpleaxis(ax)
f.legend(loc='upper center', frameon=False, ncols=2, bbox_to_anchor=(0.55,1.1))
# plt.savefig('FFL-chains-C_elegans_fractions.svg', transparent=True)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(2,2))
utils.nx.draw_circular(motif, with_labels=True, node_color='gray', node_size=300, ax=ax)
plt.show()

In [ ]:
hm = utils.make_hypermotifs(motif, 3, [(3,1)])
hm = utils.nx.relabel_nodes(hm, {'1.3-2.1':'2.1', '2.3-3.1':'3.1'})
all_ffgs = nn_male.search_motifs(hm)

In [ ]:
conn_types = {n:0 for n in ntype_pairs}
ntype_motif = {m:{n:0 for n in ntype} for m in hm.nodes}
for sub in all_ffgs:
    for motif_edge, network_edge in sub.items():
        if all(n.type in ntype for n in network_edge):
            for m,n in zip(motif_edge, network_edge):
                ntype_motif[m][n.type] +=1
            conn = tuple(sorted([network_edge[0].type, network_edge[1].type]))
            conn_types[conn] +=1

In [ ]:
pie_division = {m: [ntype_motif[m][n]/sum(ntype_motif[m].values()) for n in ntype] for m in hm.nodes}
piesize=0.09
pos = utils.nx.kamada_kawai_layout(hm)
color_dict = {p:facecolors for p in pos.keys()}
alpha_dict = {p:1 for p in pos.keys()}

f, ax = plt.subplots(figsize=(6,6), layout='constrained')
utils.nx.draw_networkx_edges(hm, pos=pos, node_size=1200, connectionstyle='arc3', arrowstyle='->')
for n,p in pos.items():
    utils.plot_pie(n=n, center=p, ax=ax, color_dict=color_dict, alpha_dict=alpha_dict, pie_division=pie_division[n], piesize=piesize)
# utils.nx.draw_networkx_labels(hm, pos=pos)
ax.set_xlim(-1, 1)
ax.set_ylim(-1, 1)
ax.set_aspect('equal')
ax.axis('off')  # Optionally turn off axis
plt.savefig('motif_ntype_ffchain_male.svg', transparent=True)


In [ ]:
conn_ligs = {}
all_ntrs = []
putative_lens = {}
motif_conns = {}
for sub in all_ffgs:
    for motif_edge, network_edge in sub.items():
        if not motif_edge in conn_ligs:
            conn_ligs[motif_edge] = []
            putative_lens[motif_edge] = []
            motif_conns[motif_edge] = []
        neuron_1, neuron_2 = network_edge[0], network_edge[1]
        conns = nn_male.connections_between(neuron_1, neuron_2, directed=True)
        for e, conn in conns.items():
            if conn.edge_type=='chemical-synapse':
                motif_conns[motif_edge].append(conn)
                conn_edges = ['-'.join(k) for k in conn.putative_neurotrasmitter_receptors if (isinstance(k[0], str) and isinstance(k[1], str))]
                putative_lens[motif_edge].append(len(conn_edges))
                conn_ligs[motif_edge].append(conn_edges)
                all_ntrs+= conn_edges

In [ ]:
color_nt = {'Ach': 'lightgreen', 'Dop':'navy', 'Glu': 'darkorange', 'GAB': 'crimson', 'Ser': 'k'}
all_labels = sorted(set(all_ntrs))
color = [color_nt[m[:3]] for m in all_labels]
fig, ax = plt.subplots(nrows=len(conn_ligs.keys()), sharex=True, sharey=True, figsize=(30,12), layout='constrained')
for j,e in enumerate(sorted(conn_ligs.keys())):
    nums = []
    for k in all_labels:
        nums.append(sum([1/len(conn_ligs[e][i]) if k in conn_ligs[e][i] else 0 for i in range(len(conn_ligs[e]))])/len(conn_ligs[e]))
    ax[j].bar(all_labels, nums, color=color)
    utils.simpleaxis(ax[j])
    ax[j].set_title(e, fontsize="xx-large")
    ax[j].set_yticks([0,0.1,0.2], labels=[0,0.1,0.2], fontsize="xx-large")
plt.xticks(rotation=45, ha='right', fontsize="xx-large")
fig.supylabel("Fraction of edges with predicted pair", fontsize='xx-large')
fig.supxlabel("Predicted neurotransmitter-receptor pair", fontsize='xx-large')
plt.savefig("Motif-FFLoop-3-chain_male.svg", transparent=True)
plt.show()
plt.close()
    # print(sum([1 if k in conn_ligs[e][i] else 0 for i in range(len(conn_ligs[e]))])/len(conn_ligs[e]))
    # conn_ligs[e])
# conn_ligs[motif_edge]

In [ ]:
num_graphs = {}
mappings = [(2,1), (3,1)]
max_chain_length = 6
nrgraphs = 50
graph_specs = {}
for mapping in mappings:
    num_graphs[mapping] = {}
    graph_specs[mapping] = {}
    for l in range(1,max_chain_length+1):
        num_graphs[mapping][l] = []
        graph_specs[mapping][l] = {'conn_type':[], 'ntype':[]}
        hm = utils.make_hypermotifs(motif, l, [mapping])
        for g in range(nrgraphs):
            nnr = utils.randomize_graph(nn_male)
            all_ffgs = nnr.search_motifs(hm)
            num_graphs[mapping][l].append(len(all_ffgs))

            conn_types = {n:0 for n in ntype_pairs}
            ntype_motif = {m:{n:0 for n in ntype} for m in hm.nodes}
            for sub in all_ffgs:
                for motif_edge, network_edge in sub.items():
                    if all(n.type in ntype for n in network_edge):
                        for m,n in zip(motif_edge, network_edge):
                            ntype_motif[m][n.type] +=1
                        conn = tuple([network_edge[0].type, network_edge[1].type])
                        conn_types[conn] +=1
            graph_specs[mapping][l]['conn_type'].append(copy.deepcopy(conn_types))
            graph_specs[mapping][l]['ntype'].append(copy.deepcopy(ntype_motif))

In [ ]:
motif_fractions = {}
for mapping in mappings:
    motif_fractions[mapping] = {}
    for l in np.arange(1,max_chain_length+1):
        motif_fractions[mapping][l] = np.array(num_graphs[mapping][l])/(np.array(num_graphs[mappings[0]][l]) + np.array(num_graphs[mappings[1]][l]))

In [ ]:
colors = ['gray', 'purple']
zscore_sig_thres = 2.58
f, ax = plt.subplots(figsize=(2.5,2.5), layout='constrained')
for m, mapping in enumerate(mappings):
    ngraphs_mu = np.mean([motif_fractions[mapping][l] for l in np.arange(1,max_chain_length+1)], axis=1)
    ngraphs_sigma = np.std([motif_fractions[mapping][l] for l in np.arange(1,max_chain_length+1)], axis=1)
    
    ax.errorbar(np.arange(1,max_chain_length+1), ngraphs_mu, yerr = ngraphs_sigma, color=colors[m], alpha=0.5, linestyle='--')
    ax.scatter(np.arange(1,len(num_graphs[mapping])+1),motif_fractions_actual[mapping] , color=colors[m], label=mapping)
    ax.plot(np.arange(1,len(num_graphs[mapping])+1),motif_fractions_actual[mapping] , color=colors[m])

    zscore_sig = np.where(np.abs([(motif_fractions_actual[mapping][l-1]- ngraphs_mu[l-1])/ngraphs_sigma[l-1] for l in np.arange(1,max_chain_length+1)])>zscore_sig_thres)[0]
    for x in zscore_sig:
        ax.text(x=x+1, y=0.8, s="**", color='k', ha='center')

    # ax.scatter(np.arange(1,len(num_graphs[mapping])+1), ngraphs_mu, color=colors[m], label=mapping)
ax.set_xticks(np.arange(1,max_chain_length+1))
ax.set_yticks((0,0.5,1))
# ax.yaxis.major.formatter._useMathText = True
ax.set_ylabel("Fraction subnetworks with motif")
ax.set_xlabel("# chained FFLs")
utils.simpleaxis(ax)
f.legend(loc='upper left', frameon=False, bbox_to_anchor=(0.15,1.05))
plt.savefig('FFL-chains-C_elegans_male_randomized_actual_edges_fractions_new.svg', transparent=True)
plt.show()